## Google Analytics Customer Revenue Prediction

In [24]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize

from scipy import stats
from scipy.stats import norm, ttest_ind, chisquare, chi2_contingency, chi2

from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn import linear_model
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression


In [25]:
filePath = 'partTest.csv'
untreated = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [26]:
df_test = pd.read_csv(filePath, converters={feature: json.loads for feature in untreated}, dtype={'fullVisitorId': 'str'}, nrows=None)

In [27]:
for i in untreated:
    normalize_df = json_normalize(df_test[i])
    normalize_df.columns = ["{}.{}".format(i,j) for j in normalize_df.columns]
    df_test = df_test.drop(i, axis=1).merge(normalize_df, right_index=True, left_index=True)

In [28]:
df_test.columns

Index([u'channelGrouping', u'date', u'fullVisitorId', u'sessionId',
       u'socialEngagementType', u'visitId', u'visitNumber', u'visitStartTime',
       u'device.device', u'geoNetwork.geoNetwork', u'totals.totals',
       u'trafficSource.adContent',
       u'trafficSource.adwordsClickInfo.adNetworkType',
       u'trafficSource.adwordsClickInfo.criteriaParameters',
       u'trafficSource.adwordsClickInfo.gclId',
       u'trafficSource.adwordsClickInfo.isVideoAd',
       u'trafficSource.adwordsClickInfo.page',
       u'trafficSource.adwordsClickInfo.slot', u'trafficSource.campaign',
       u'trafficSource.isTrueDirect', u'trafficSource.keyword',
       u'trafficSource.medium', u'trafficSource.referralPath',
       u'trafficSource.source'],
      dtype='object')

In [29]:
df_test

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.device,geoNetwork.geoNetwork,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,6.16787E+18,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'(not set)', u'networkDomain': u'my...",...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
1,Organic Search,20171016,6.43698E+17,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'Zaragoza', u'networkDomain': u'rim...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20171016,6.05938E+18,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'not available in demo dataset', u'...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20171016,2.37672E+18,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'Mountain View', u'networkDomain': ...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,2.31454E+18,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'San Jose', u'networkDomain': u'(no...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
5,Organic Search,20171016,4.13304E+18,4133039884103392367_1508186358,Not Socially Engaged,1508186358,1,1508186358,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'not available in demo dataset', u'...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
6,Organic Search,20171016,4.32048E+18,4320478850207397557_1508203650,Not Socially Engaged,1508203650,1,1508203650,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'not available in demo dataset', u'...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
7,Organic Search,20171016,5.87644E+18,5876438247590157131_1508184397,Not Socially Engaged,1508184397,1,1508184397,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'not available in demo dataset', u'...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
8,Organic Search,20171016,5.14591E+17,0514591268737702944_1508189652,Not Socially Engaged,1508189652,6,1508189652,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'Lisbon', u'networkDomain': u'pt', ...",...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
9,Organic Search,20171016,6.43057E+18,6430567031531677212_1508175502,Not Socially Engaged,1508175502,1,1508175502,{u'mobileDeviceBranding': u'not available in d...,"{u'city': u'Zaragoza', u'networkDomain': u'rim...",...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
